<center>
<h1>

**B. Normalizacion de Datos**

</h1>
<center>

El proyecto trata de descubrir clientes potenciales para una distribuidora de material industrial.

En esta etapa:

El objetivo es: Mapeo de Productos

Se tienen los siguientes datos crudos:
* reportes de ventas anualizadas de 2020 a 2023


---

## 1. Integridad de la base de datos

In [449]:
import pandas as pd
import MungingOps as mo
import TextTools as tt

In [450]:
VENTAS_NORMAL = pd.read_pickle("NORMALIZED_VENTAS.data")
VENTAS_NORMAL.head(1)

,Codigo,Nombre,Cantidad,Unidad,Neto,Descuento,DescuentoNeto,Impuesto,Total,Year,UserRfc,ClientID,ClientName
0,25037,CASCO DE SEGURIDAD AMARILLO,20.0,PZA,1260.0,0.0,1260.0,100.8,1360.8,2020,GUOR86061511A,CL001,ABASTECEDORA DE FIERRO Y ACERO SA DE CV


In [451]:
COMPRAS_NORMAL = pd.read_pickle("NORMALIZED_COMPRAS.data")
COMPRAS_NORMAL.head(1)

,Codigo,Nombre,Cantidad,Unidad,Neto,Descuento,DescuentoNeto,Impuesto,Total,Year,UserRfc,ProveedorID,ProveedorName
0,11184,LIGA,135.0,LIB,10462.5,0.0,10462.5,1674.0,12136.5,2020,GUOR86061511A,P043,ALMA ANGELINA PANTOJA GOMEZ


Para cruzar los datos solo se necesitan las siguientes variables:
* Codigo
* nombre
* Unidad

In [452]:
Rows = ["Codigo","Nombre","Unidad"]

In [453]:
COMPRAS_DATA = pd.DataFrame()
for i in Rows:
    COMPRAS_DATA[i] = COMPRAS_NORMAL[i]

In [454]:
df=COMPRAS_DATA.drop_duplicates(subset=['Nombre'])
del COMPRAS_DATA
COMPRAS_DATA=df#.drop_duplicates(subset=['Codigo'])
del df
COMPRAS_DATA

,Codigo,Nombre,Unidad
0,11184,LIGA,LIB
1,LIM110,PANUELO FACIAL KLEENEX 90HS C/72 CAJITAS,CAJ
2,TPE-OFFI,TAEIPERA DE OFICINA,PZA
3,25218,CREMA DESENGRASANTE PARA MANOS FAST ORANGE,GAL
4,TIN19,TINNER 19 LTS,CU
...,...,...,...
2307,LDPE-044,LLDPE SABIC 318BJA,KG
2328,SB7778,ANTIBLOCK,KG
2347,SS-RMG-MD,GUANTE DE ACERO INOXIDABLE M,PZA
2348,SS-RMG-SM,GUANTE DE ACERO INOXIDABLE S,PZA


In [455]:
COMPRAS_DATA.sort_values(by='Nombre', ascending=False, inplace=True)

In [456]:
VENTAS_DATA = pd.DataFrame()
for i in Rows:
    VENTAS_DATA[i] = VENTAS_NORMAL[i]

In [457]:
df=VENTAS_DATA.drop_duplicates(subset=['Nombre'])
del VENTAS_DATA
VENTAS_DATA=df#.drop_duplicates(subset=['Codigo'])
del df
VENTAS_DATA

,Codigo,Nombre,Unidad
0,25037,CASCO DE SEGURIDAD AMARILLO,PZA
1,ELN-01-26,BOTA INDUSTRIAL EVOLUTION,PAR
7,JYR-8252XL-3C,FAJA DE SEGURIDAD CON 3ER CINTO JYRSA XXL,PZA
8,JYR-825L-3C,FAJA DE SEGURIDAD CON 3ER CINTO JYRSA L,PZA
9,JYR-825M-3C,FAJA DE SEGURIDAD CON 3ER CINTO JYRSA M,PZA
...,...,...,...
12019,GT-0009-S,GUANTE ANTICORTE CON RECUBRIMIENTO DE NITRILO S,DOC
12023,SS-RMG-MD,GUANTE DE ACERO INOXIDABLE M,PZA
12024,SS-RMG-SM,GUANTE DE ACERO INOXIDABLE S,PZA
12043,BP35-00065-K,BOLSA NEGRA PARA BASURA 65 GAL KILEADO,KG


In [458]:
VENTAS_DATA.sort_values(by='Nombre', ascending=False, inplace=True)

In [459]:
VENTAS_DATA.head(1)

,Codigo,Nombre,Unidad
2275,H-4453,X-ACTO PRECISION INSTRUMENTS,CAJ


verificando la integridad de los articulos

In [460]:
COMPRAS_DATA[ COMPRAS_DATA['Codigo'] == 'LIM104' ]

,Codigo,Nombre,Unidad
47,LIM104,SERVILLETA TIPO RESTAURANT C/500 PIEZAS,PAQ


In [461]:
VENTAS_DATA[ VENTAS_DATA['Codigo'] == 'LIM104' ]

,Codigo,Nombre,Unidad
1306,LIM104,SERVILLETA TIPO RESTAURANT C/500 PIEZAS,PAQ


In [462]:
print('Existe un diferencial de productos, se ofrecen',VENTAS_DATA.shape[0],'productos y se compran',COMPRAS_DATA.shape[0],'\n existe un diferencial de',COMPRAS_DATA.shape[0]-VENTAS_DATA.shape[0], 'productos')

Existe un diferencial de productos, se ofrecen 1092 productos y se compran 818 
 existe un diferencial de -274 productos


### Busqueda de Patrones

Se intentara crear una relacion para fusionar los dataset mediante la asignacion de propiedades de Jaccard, el primer paso es determinar los articulos que no existen en la compra pero si en la venta

In [463]:
#Variables = ['Codigo','Nombre','Unidad']

La aproximacion mas correcta es buscar por jaccard que articulos de VENTAS_DATA corresponden a COMPRAS_DATA

In [464]:
# Flexibility, rapid responce:
# This is not an oraculus, you not magicaly obtain the responce
# Manipulate the number to manipulate the result
#
#
# Flexibility, Razonable explication:
# Flexibility refers to the range of bad response can be accepted;
# this range is a quasy fuzzy response, not a binary logic
# then the Jaccard Distance say this string is most ok or not ok,
# traditionaly the jaccard distance in 0.8 is acceptable, but
# sometimes is necesary more low number to obtain a responce
#
Flexibility = 0.85

In [465]:
# Exceptions Vector
ExceptionTitles = pd.DataFrame()
ExceptionTitles = VENTAS_DATA.copy()
df=COMPRAS_DATA
#
# Calculate the Jaccard Distance by each VENTAS_DATA for each COMPRAS_DATA
# Select the maximum Jaccard
# if Jaccard is > Flexibility the make a Suggestion
# si no pues no -0_o-
#
Correction=[];Suggested=[]
#
for i in ExceptionTitles['Codigo']:
    ComparisonScore=[]
    ComparisonTitle=[]
    ComparisonCode=[]
    for j in df['Codigo']:
        ComparisonScore.append(tt.JaccardDistance( i,j ) )
        ComparisonCode.append( df[ df['Codigo'] == j ][ 'Codigo' ].to_list()[0] )
        ComparisonTitle.append( df[ df['Codigo'] == j ][ 'Nombre' ].to_list()[0] )
        Score = max( ComparisonScore )
        Code = ComparisonCode[ ComparisonScore.index( max( ComparisonScore ) ) ]
        Title = ComparisonTitle[ ComparisonScore.index( max( ComparisonScore ) ) ]
        #MaxJaccard = JaccardValues[1]
    if Score > Flexibility:
        Suggested.append( Code )
        Correction.append( Title )
    elif Score < Flexibility:
        Suggested.append('')
        Correction.append('')
    else:
        Suggested.append('')
        Correction.append('')

In [466]:
# Add the results to the dataframe
#
ExceptionTitles['CodigobyCodigo'] = Suggested
ExceptionTitles['TitlebyCodigo'] = Correction

In [467]:
ExceptionTitles.head()

,Codigo,Nombre,Unidad,CodigobyCodigo,TitlebyCodigo
2275,H-4453,X-ACTO PRECISION INSTRUMENTS,CAJ,,
140,R-WD,WIPER DESECHABLE CON 120 HOJAS,ROL,R-WD,WIPER DESECHABLE CON 120 HOJAS
6497,WIP-NW-CP-68-9,WIPER CELULOSA POLIESTER 9 X9 C/10 PAQEUTE DE ...,CAJ,,
449,PRT-2091NV,WIPER 100% POLIESTER 9X9 C/50,PAQ,PRT-2091NV,WIPER 100% POLIESTER 9X9 C/50
456,WD10116,WD-40 AEROSOL 16 OZ,PZA,WD10116,WD-40 AEROSOL 16 OZ


In [468]:
NullTitles = ExceptionTitles[ ExceptionTitles['CodigobyCodigo'] == '' ]
NullTitles

,Codigo,Nombre,Unidad,CodigobyCodigo,TitlebyCodigo
2275,H-4453,X-ACTO PRECISION INSTRUMENTS,CAJ,,
6497,WIP-NW-CP-68-9,WIPER CELULOSA POLIESTER 9 X9 C/10 PAQEUTE DE ...,CAJ,,
8516,V-T4,VASO TRANSPARENTE,CAJ,,
2985,VT-12,VASO TIPO TERMICO No 12 C/1000 REYMA,CAJ,,
10251,T6 -400,TUBULAR 6TRANSPARENTE CAL.400 ROLLO DE 15 KG,ROL,,
...,...,...,...,...,...
9373,LIM017,BASTON INSTITUCIONAL DE METAL CHICO,PZA,,
3573,ANTICIPO,ANTICIPO DEL BIEN O SERVICIO,NaN,,
12076,14318,ADAPTADOR DE PROTECTOR FACIAL PARA CASCO,PZA,,
1434,EWR-9103,ACCESORIO CAIMAN CROMADO,PZA,,


In [470]:
NonNullTitles = ExceptionTitles[ ExceptionTitles['CodigobyCodigo'] != '' ]
NonNullTitles

,Codigo,Nombre,Unidad,CodigobyCodigo,TitlebyCodigo
140,R-WD,WIPER DESECHABLE CON 120 HOJAS,ROL,R-WD,WIPER DESECHABLE CON 120 HOJAS
449,PRT-2091NV,WIPER 100% POLIESTER 9X9 C/50,PAQ,PRT-2091NV,WIPER 100% POLIESTER 9X9 C/50
456,WD10116,WD-40 AEROSOL 16 OZ,PZA,WD10116,WD-40 AEROSOL 16 OZ
36,VT-08,VASO TIPO TERMICO No 8 C/1000 REYMA,CAJ,VT-08,VASO TIPO TERMICO No 8 C/1000 REYMA
2888,VT-10,VASO TIPO TERMICO No 10C/1000 REYMA,CAJ,VT-10,VASO TIPO TERMICO No 10 C/1000 REYMA
...,...,...,...,...,...
35,NAEH009,ABRINTALLADOR DE MADERA WIESE,PZA,NAEH009,ABRINTALLADOR DE MADERA WIESE
6889,44232,ABRAZADERAS REFORZADAS DE ACERO INOXIDABLE 3/8...,PAQ,44232,ABRAZADERAS REFORZADAS DE ACERO INOXIDABLE 3/8...
6888,44231,ABRAZADERA REFORZADAS DE ACERINOXIDABLE,PZA,13342,BROCHA 1 MANGO DE MADERA
6890,44236,ABRAZADERA REFORZADA GRANDE,PZA,44236,ABRAZADERA REFORZADA GRANDE


In [471]:
NullTitles.to_csv('NullTitles.csv')

In [472]:
NonNullTitles.to_csv('NonNullTitles.csv')

---







.